In [1]:
%config Completer.use_jedi = False
import pandas as pd
import re
import collections
from matplotlib import pyplot as plt
import seaborn as sns
import pymysql
import random
import json

**함수**

In [2]:
def processing(content):
    result = re.sub(r'[a-zA-Z가-힣]+뉴스', '', str(content))
    result = re.sub(r'[a-zA-Z가-힣]+ 뉴스', '', result)
    result = re.sub(r'[a-zA-Z가-힣]+newskr', '', result)
    result = re.sub(r'[a-zA-Z가-힣]+Copyrights', '', result)    
    result = re.sub(r'[a-zA-Z가-힣]+ Copyrights', '', result)
    result = re.sub(r'\s+Copyrights', '', result)
    result = re.sub(r'[a-zA-Z가-힣]+com', '', result)
    result = re.sub(r'[가-힣]+ 기자', '', result)    
    result = re.sub(r'[가-힣]+기자', '', result)
    result = re.sub(r'[가-힣]+ 신문', '', result)    
    result = re.sub(r'[가-힣]+신문', '', result)
    result = re.sub(r'데일리+[가-힣]', '', result)
    result = re.sub(r'[가-힣]+투데이', '', result)    
    result = re.sub(r'[가-힣]+미디어', '', result)
    result = re.sub(r'[가-힣]+ 데일리', '', result)
    result = re.sub(r'[가-힣]+데일리', '', result)
    result = re.sub(r'[가-힣]+ 콘텐츠 무단', '', result)
    result = re.sub(r'전재\s+변형', '전재', result)
    result = re.sub(r'[가-힣]+ 전재', '', result)    
    result = re.sub(r'[가-힣]+전재', '', result)
    result = re.sub(r'[가-힣]+배포금지', '', result)
    result = re.sub(r'[가-힣]+배포 금지', '', result)
    result = re.sub(r'\s+배포금지', '', result)
    result = re.sub(r'\s+배포 금지', '', result)    
    result = re.sub(r'[a-zA-Z가-힣]+.kr', '', result)
    result = re.sub(r'/^[a-z0-9_+.-]+@([a-z0-9-]+\.)+[a-z0-9]{2,4}$/', '', result)
    result = re.sub(r'[\r|\n]', '', result)
    result = re.sub(r'\[[^)]*\]', '', result)
    result = re.sub(r'\([^)]*\)', '', result)
    result = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z0-9]', '', result)
    result = result.replace('뉴스코리아','').replace('및','').replace('Copyright', '').replace('저작권자','').replace('ZDNET A RED VENTURES COMPANY','')    
    result = result.strip()

    return result

In [3]:
def category_add(content):
    labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']    
    k_i_dict = {k: i for i, k in enumerate(labels)}
    
    category = []
    for key, value in k_i_dict.items():
        if str(key) in content:
            category.append(value)

    if not category:
        category=[29]
    
    return category

In [4]:
def categorytext_add(content):
    labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']    
    k_i_dict = {k: i for i, k in enumerate(labels)}
    
    category = ""
    
    for key, value in k_i_dict.items():
        if str(key) in content:
            if category:
                category += ","            
            category += str(value)
            
    if not category:
        category="29"
    
    return category

In [5]:
def label_modify(category):
    if len(category) > 3:
        random.shuffle(category)
        category = category[:3]

    return category

In [6]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

def load_jsonl(input_path) -> list:
    """
    Read list of objects from a JSON lines file.
    """
    data = []
    with open(input_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line.rstrip('\n|\r')))
    print('Loaded {} records from {}'.format(len(data), input_path))
    return data

**Labeling**

In [16]:
conn = pymysql.connect(
    user="root",
    passwd="Illunex123!",
    db="portal_news_scraper",
    host="172.30.1.44",
    port=3307,
    charset="utf8",
    use_unicode=True,
)
cursor = conn.cursor(pymysql.cursors.DictCursor)

# query = """select * from portal_news_scraper.portal_news where portal='naver' and kind not in ('200106', '200107')
#             limit 500000;"""
query = """select * from portal_news_scraper.portal_news where portal='daum' and kind not in ('200206', '200207')
            limit 500000;"""

cursor.execute(
    query
)
data = pd.DataFrame(cursor.fetchall())

In [17]:
data['content'] = data.content.apply(processing) 

In [18]:
data['content'][17]

'8월 예방접종 계획에서는 우선 만50세에서 54세까지 연령층이 맞을 백신의 종류가 확정돼야 합니다당초 50대는 모두 모더나였으나 50대 후반은 대부분 화이자로 바뀌고 접종 일정도 수차례 조정됐습니다접종 물량이 모자라 화이자 백신은 과학적 근거 없이 1 2차 접종 간격을 3주에서 4주로 늘리기까지 했습니다모더나 백신의 국내 도입이 차질을 빚었기 때문입니다하지만 정부 발표대로 8월부터 모더나 공급이 원활해지면 오는 16일부터 접종을 시작할 50대 초반은 모더나를 맞을 가능성이 높습니다모더나는 8월에만 천만 회분 가까이 공급되는 것으로 알려졌습니다50대 1차 접종이 8월 28일에 마무리되면 만18세에서 49세까지 연령층에 대한 접종이 본격화할 것으로 보입니다천7백만 명에 이르는 이 연령대는 화이자를 주력으로 모더나를 같이 활용할 계획이었으나 모더나의 비중이 좀 더 높아질 전망입니다50대 접종에 화이자를 미리 가져다 썼기 때문입니다다만 백신 수급이 여전히 불안하다는 점에서 각 연령대가 맞을 백신의 종류와 접종 일정 등 구체적인 사항은 주 단위로 안내될 것으로 보입니다모더나는 자체 생산 설비를 갖고 있지 않아 생산 차질 우려가 여전히 남아있고 4천만 회분 도입이 예정된 노바백스는 아직 허가도 신청하지 않은 상태입니다여러 변수가 남아있지만 정부는 9월까지 전 국민의 70인 3천6백만 명에 대한 백신 1차 접종 목표를 달성할 수 있을 것으로 자신하고 있습니다현재까지 백신 1차 접종은 천838만 명으로 전체 인구의 35 목표치와 비교하면 50를 넘어섰습니다YTN 박홍구입니다YTN 박홍구  당신의 제보가 뉴스가 됩니다 YTN은 여러분의 소중한 제보를 기다립니다 wwwytncokr저작권자 YTN  YTN plus 무단 전재재배포 금지'

In [9]:
print(data.content)

0         1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...
1         서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...
2         손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...
3         장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...
4         LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...
                                ...                        
499995    아버지를 닮은 아버지 같은 해군 간부가 되겠다우석대 진천캠퍼스 군사안보학과 김해나 ...
499996    평택시 기록관은 시민의 생활상과 추억이 담긴 민간기록물을 오는 10월 29일까지 수...
499997    국립낙동강생물자원관 연구진경북 상주시에 있는 환경부 산하 국립낙동강생물자원관이 최근...
499998    총 상금 1200만 원 규모 8개 과채 소재 사진과 동영상 카드뉴스로 응모전북농협전...
499999    김제시청   전북 김제시는 대한민국 국회의원 의정 대상 시상위원회 등이 주관하는 2...
Name: content, Length: 500000, dtype: object


In [10]:
data = data.dropna()

In [11]:
data.count()

id             460848
url            460848
title          460848
content        460848
create_date    460848
kind           460848
url_md5        460848
image_url      460848
portal         460848
category       460848
media          460848
dtype: int64

In [12]:
data['category'] = data.content.apply(category_add)

In [13]:
data.head(20)

,id,url,title,content,create_date,kind,url_md5,image_url,portal,category,media
0,121,https://news.naver.com/main/read.naver?mode=LS...,“日 의존도 낮춰보자” 핵심품목 국내기술 대체 노력이 결실로,1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...,2021-07-27 03:02:00,200105,5dca026c68c80e654f1e8b5dc9e40f1b,https://imgnews.pstatic.net/image/020/2021/07/...,naver,"[3, 5, 9, 10, 20, 23]",
1,122,https://news.naver.com/main/read.naver?mode=LS...,서창호 교수 IEEE 젊은 과학자상,서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...,2021-07-27 03:02:00,200105,bcd1668e19216c906aa912d27853a80a,https://imgnews.pstatic.net/image/020/2021/07/...,naver,"[0, 9, 21]",
2,123,https://news.naver.com/main/read.naver?mode=LS...,디디추싱 투자한 손정의도 4조6000억 손실,손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...,2021-07-27 03:02:00,200105,0d045af48634cc949545bebfbc98d7b4,no image,naver,[5],
3,124,https://news.naver.com/main/read.naver?mode=LS...,“크래프톤엔 누구도 안가는 길 가는 똘끼 유전자 있다”,장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...,2021-07-27 03:01:00,200105,3c2492aff97ab3d8a69f37cb2e772dc5,https://imgnews.pstatic.net/image/023/2021/07/...,naver,[5],
4,125,https://news.naver.com/main/read.naver?mode=LS...,"[Biz & Now] LG디스플레이, ESG 차원 신재생에너지 구매",LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...,2021-07-27 12:04:00,200105,0dd09cc92dfe820588a6e3ac61ec2b93,no image,naver,"[3, 9]",
5,127,https://news.naver.com/main/read.naver?mode=LS...,"“게임을 다양한 미디어로 확장, 지속가능한 회사 만들 것”",고평가 논란 크래프톤 비전 발표내달 IPO예상 시총 최대 24조똘끼 DNA로 기회 ...,2021-07-27 12:04:00,200105,643a050ed1a1ec8ad2180920afc525ea,no image,naver,[17],
6,128,https://news.naver.com/main/read.naver?mode=LS...,서창호 교수 ‘제임스 매시상’ 수상,카이스트 전기전자공학부 서창호 교수가 박사학위 취득 후 10년 이내 젊은 학자에게 ...,2021-07-26 08:12:00,200105,8e3a14acc114c106ead22a757eeafffc,no image,naver,"[0, 9, 21]",
7,130,https://news.naver.com/main/read.naver?mode=LS...,"올해 IPO 최대어 크래프톤... 장병규 의장 ""글로벌 게임시장에 접근할 수 있는 투자""",8월 23일 일반청약 간담회 열어고평가 논란 등 시장 우려에 조목조목 반박장병규 크...,2021-07-26 08:01:00,200105,aca29717d8c5386e25891aa8ffb3f9eb,https://imgnews.pstatic.net/image/469/2021/07/...,naver,"[5, 9]",
8,241,https://news.naver.com/main/read.naver?mode=LS...,3~5명 규모로 공공 과외… 교원확보 등 불투명해 효과 미지수,교육부 교육회복 방안 발표방과후방학중 맞춤형 지도수석교사 등이 학습 컨설팅초중고 ...,2021-07-29 12:01:00,200102,a99050b3a72dd5e24df5c3ba79c438fb,https://imgnews.pstatic.net/image/021/2021/07/...,naver,"[0, 21]",
9,242,https://news.naver.com/main/read.naver?mode=LS...,"김홍빈 대장 장례, 내달 2~6일 광주 염주 체육관서 ‘산악인장’",히말라야 14좌를 완등하고 하산 중 실종된 김홍빈 대장의 장례가 광주에서 산악인장으...,2021-07-29 11:55:00,200102,f18b6c6c7e9893988fb79e78ae860087,https://imgnews.pstatic.net/image/016/2021/07/...,naver,[29],


**count check**

In [14]:
# labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']
# k_i_dict = {k: i for i, k in enumerate(labels)}

# tolist = data.category.tolist()
# tolist = sum(tolist, [])
# for key, value in k_i_dict.items():
#     a = tolist.count(value)
#     print(key + " : " + str(a))

인공지능 : 9817
로봇 : 3867
스마트팜 : 469
에너지 : 15742
서버 : 3001
투자 : 44129
정부지원 : 406
증강현실 : 1259
이동수단 : 315
개발 : 59245
통신 : 44557
과학 : 19849
드론 : 2130
블록체인 : 3098
핀테크 : 1882
커머스 : 4891
여행 : 10169
미디어 : 9806
헬스케어 : 2979
의약 : 7373
식품 : 15140
교육 : 35178
직업 : 4911
경제 : 83120
광고 : 6793
제약 : 8440
O2O : 187
뷰티 : 2261
부동산 : 19396
etc : 224233


**drop etc**

In [15]:
data['categorytext'] = data.content.apply(categorytext_add)

In [16]:
data['categorytext'].head(20)

0     3,5,9,10,20,23
1             0,9,21
2                  5
3                  5
4                3,9
5                 17
6             0,9,21
7                5,9
8               0,21
9                 29
10                29
11                29
12                29
13                21
14                29
15              3,21
16                29
17         5,9,16,21
18                29
19                29
Name: categorytext, dtype: object

In [17]:
data

,id,url,title,content,create_date,kind,url_md5,image_url,portal,category,media,categorytext
0,121,https://news.naver.com/main/read.naver?mode=LS...,“日 의존도 낮춰보자” 핵심품목 국내기술 대체 노력이 결실로,1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...,2021-07-27 03:02:00,200105,5dca026c68c80e654f1e8b5dc9e40f1b,https://imgnews.pstatic.net/image/020/2021/07/...,naver,"[3, 5, 9, 10, 20, 23]",,"3,5,9,10,20,23"
1,122,https://news.naver.com/main/read.naver?mode=LS...,서창호 교수 IEEE 젊은 과학자상,서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...,2021-07-27 03:02:00,200105,bcd1668e19216c906aa912d27853a80a,https://imgnews.pstatic.net/image/020/2021/07/...,naver,"[0, 9, 21]",,"0,9,21"
2,123,https://news.naver.com/main/read.naver?mode=LS...,디디추싱 투자한 손정의도 4조6000억 손실,손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...,2021-07-27 03:02:00,200105,0d045af48634cc949545bebfbc98d7b4,no image,naver,[5],,5
3,124,https://news.naver.com/main/read.naver?mode=LS...,“크래프톤엔 누구도 안가는 길 가는 똘끼 유전자 있다”,장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...,2021-07-27 03:01:00,200105,3c2492aff97ab3d8a69f37cb2e772dc5,https://imgnews.pstatic.net/image/023/2021/07/...,naver,[5],,5
4,125,https://news.naver.com/main/read.naver?mode=LS...,"[Biz & Now] LG디스플레이, ESG 차원 신재생에너지 구매",LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...,2021-07-27 12:04:00,200105,0dd09cc92dfe820588a6e3ac61ec2b93,no image,naver,"[3, 9]",,"3,9"
...,...,...,...,...,...,...,...,...,...,...,...,...
499995,1860616,https://news.naver.com/main/read.naver?mode=LS...,"'천안함 용사 딸' 우석대 김해나, 해군장교 선발",아버지를 닮은 아버지 같은 해군 간부가 되겠다우석대 진천캠퍼스 군사안보학과 김해나 ...,2021-08-20 13:20:22,200102,a1f8848f71a220a5b9850051a389c214,https://imgnews.pstatic.net/image/421/2021/08/...,naver,[21],뉴스1,21
499996,1860617,https://news.naver.com/main/read.naver?mode=LS...,"""지역정체성 확립한다""…평택시, 민간기록 수집 프로젝트 진행",평택시 기록관은 시민의 생활상과 추억이 담긴 민간기록물을 오는 10월 29일까지 수...,2021-08-20 13:20:24,200102,2697619e5248433f9925657fb1e889f7,no image,naver,[9],머니S,9
499997,1860618,https://news.naver.com/main/read.naver?mode=LS...,피부 유해세균 억제하는 신종 유산균 발견,국립낙동강생물자원관 연구진경북 상주시에 있는 환경부 산하 국립낙동강생물자원관이 최근...,2021-08-20 13:20:27,200102,0efc4aeba52d0b8ad7b16d84dccef065,https://imgnews.pstatic.net/image/011/2021/08/...,naver,[9],서울경제,9
499998,1860619,https://news.naver.com/main/read.naver?mode=LS...,"농협경제지주, '2021 우리 농산물 과채류 매력 알리기 콘텐츠 챌린지' 공모전 개최",총 상금 1200만 원 규모 8개 과채 소재 사진과 동영상 카드뉴스로 응모전북농협전...,2021-08-20 13:20:30,200102,5a6d7bb8c164cbbc0d8dc8e170bd14b1,no image,naver,[29],프레시안,29


In [22]:
index = data['categorytext'].isin(['29']) & (data['id'] <= 1800000)
print(index)
data2 = data[index == False]

0         False
1         False
2         False
3         False
4         False
          ...  
499995    False
499996    False
499997    False
499998    False
499999    False
Length: 460848, dtype: bool


In [23]:
data2['categorytext'].head(20)

0     3,5,9,10,20,23
1             0,9,21
2                  5
3                  5
4                3,9
5                 17
6             0,9,21
7                5,9
8               0,21
13                21
15              3,21
17         5,9,16,21
21                23
22                28
27          3,5,9,11
29                28
31               0,5
32                28
34                28
35           5,10,23
Name: categorytext, dtype: object

In [24]:
data2.count()

id              251513
url             251513
title           251513
content         251513
create_date     251513
kind            251513
url_md5         251513
image_url       251513
portal          251513
category        251513
media           251513
categorytext    251513
dtype: int64

In [25]:
labels=['인공지능', '로봇', '스마트팜', '에너지', '서버', '투자', '정부지원', '증강현실', '이동수단', '개발', '통신', '과학', '드론', '블록체인', '핀테크', '커머스', '여행', '미디어', '헬스케어', '의약', '식품', '교육', '직업', '경제', '광고', '제약', 'O2O', '뷰티', '부동산', 'etc']
k_i_dict = {k: i for i, k in enumerate(labels)}

tolist = data2.category.tolist()
tolist = sum(tolist, [])
for key, value in k_i_dict.items():
    a = tolist.count(value)
    print(key + " : " + str(a))

인공지능 : 9817
로봇 : 3867
스마트팜 : 469
에너지 : 15742
서버 : 3001
투자 : 44129
정부지원 : 406
증강현실 : 1259
이동수단 : 315
개발 : 59245
통신 : 44557
과학 : 19849
드론 : 2130
블록체인 : 3098
핀테크 : 1882
커머스 : 4891
여행 : 10169
미디어 : 9806
헬스케어 : 2979
의약 : 7373
식품 : 15140
교육 : 35178
직업 : 4911
경제 : 83120
광고 : 6793
제약 : 8440
O2O : 187
뷰티 : 2261
부동산 : 19396
etc : 14898


In [27]:
data2 = data2[['category', 'content']]

In [33]:
data2

,category,content
0,"[23, 20, 9, 3, 5, 10]",1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...
1,"[0, 9, 21]",서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...
2,[5],손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...
3,[5],장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...
4,"[3, 9]",LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...
...,...,...
499995,[21],아버지를 닮은 아버지 같은 해군 간부가 되겠다우석대 진천캠퍼스 군사안보학과 김해나 ...
499996,[9],평택시 기록관은 시민의 생활상과 추억이 담긴 민간기록물을 오는 10월 29일까지 수...
499997,[9],국립낙동강생물자원관 연구진경북 상주시에 있는 환경부 산하 국립낙동강생물자원관이 최근...
499998,[29],총 상금 1200만 원 규모 8개 과채 소재 사진과 동영상 카드뉴스로 응모전북농협전...


**label modify**

In [28]:
data['category2'] = data2.category.apply(label_modify)

In [29]:
data.rename(columns={'category':'origin_category', 'category2':'category'}, inplace=True)

In [31]:
data = data[['category', 'content']]

In [43]:
data = data.dropna()

In [45]:
data.count()

category    251513
content     251513
dtype: int64

**data save**

In [46]:
df = pd.DataFrame(data)

In [47]:
df.to_csv('/workspace/data/git_category.csv', encoding='utf-8')

In [48]:
temp_dict = [
    {"content": row['content'].strip(), "category": row['category']}
    for _, row in df.iterrows()
]

temp_dict[0]

{'content': '1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품의 부패를 방지하기 위해 산소 차단 기능이 적용된 포장재는 이전까지 일본산 소재인 에틸렌비닐알코올로만 만들어졌다 에버켐텍은 천연 단백질을 이용한 신소재로 일본의 식품포장재 시장 독점 구조를 깬 셈이다 에버켐텍이 소재부품장비 강소기업으로 평가받는 이유다2 2001년 설립된 영창케미칼은 일본의 3대 수출규제 품목 중 하나였던 포토레지스트 소재 국산화를 선도했다 반도체 공정의 필수 소재인 포토레지스트는 빛으로 회로 모양을 찍어내는 작업을 할 때 웨이퍼 위에 균일하게 도포되는 액체를 말한다 영창케미칼의 포토레지스트는 경쟁력을 인정받아 현재 삼성전자 SK하이닉스 등 국내 반도체 회사에 공급된다일본 경제산업성이 2019년 7월 고순도 불화수소 플루오린폴리이미드 포토레지스트에 대한 수출규제를 시작하면서 촉발된 소부장 기술전쟁이 2년째를 맞는다 정부는 소부장 기업들이 혁신과 정부 지원정책이 맞물려 기술 자립도가 높아졌다고 평가한다 100대 핵심 품목 대일 의존도 급감중소벤처기업부와 산업통상자원부에 따르면 최근 2년 동안 소부장 관련 100대 핵심 품목에 대한 대일 의존도는 줄고 소부장 기업의 매출은 20 넘게 증가했다 중기부가 선정한 소부장 강소기업100에 포함돼 있는 에버켐텍과 영창케미칼 등의 성과를 보면 일본의 수출규제가 소부장 기업들이 성장하는 계기가 된 셈이다소부장 기술독립의 성과는 양과 질적인 면에서 함께 나타나고 있다 시가총액이 1조 원 이 넘는 소부장 관련 중소중견기업 수는 2019년 13개에서 올해 31개로 늘었다소부장 생태계 내에서 대기업과 중소기업 간 협업 사례도 늘었다 중기부에 따르면 대기업은 생산 라인을 개방해 중소기업이 신규 기술을 검증받을 수 있도록 했다 그러면서 대기업은 중소기업이 개발한 소재를 공급받고 있다 이에 따라 지난해 1분기 소부장 상장기업의 총매출액은 2019년 1분기 대비 201 증가했다 이는 상장기업 전체 평균 매출액 증가율보다 

In [50]:
# df.to_json('/home/shpark/news_scraper/model/data/git_category.csv')
!rm -rf /workspace/data/git_category.json
dump_jsonl(temp_dict, '/workspace/data/git_category.json')

Wrote 251513 records to /workspace/data/git_category.json


In [51]:
df.head(20)

,category,content
0,"[23, 20, 9]",1강소기업의 기술독립 전쟁1 에버켐텍은 2019년 친환경 식품포장재를 개발했다 식품...
1,"[0, 9, 21]",서창호 KAIST 전기전자공학부 교수가 국제전기전자공학회 정보이론학회에서 수여하는 ...
2,[5],손정의 일본 소프트뱅크 회장의 비전펀드가 중국발 악재에 막대한 손실을 떠안게 됐다 ...
3,[5],장병규 의장 IPO 기자간담회크래프톤 IPO 온라인 간담회에서 장병규 크래프톤 의장...
4,"[3, 9]",LG디스플레이가 ESG 경영 강화 목적으로 한국전력의 녹색프리미엄 제도를 활용해 태...
5,[17],고평가 논란 크래프톤 비전 발표내달 IPO예상 시총 최대 24조똘끼 DNA로 기회 ...
6,"[0, 9, 21]",카이스트 전기전자공학부 서창호 교수가 박사학위 취득 후 10년 이내 젊은 학자에게 ...
7,"[5, 9]",8월 23일 일반청약 간담회 열어고평가 논란 등 시장 우려에 조목조목 반박장병규 크...
8,"[0, 21]",교육부 교육회복 방안 발표방과후방학중 맞춤형 지도수석교사 등이 학습 컨설팅초중고 ...
13,[21],롯데건설 손승익 팀장과 율하모니 이윤정 대표가 업무협약을 체결 후 기념 촬영하고 있...


In [52]:
df.count()

category    251513
content     251513
dtype: int64

**데이터 확인**

In [22]:
predata = pd.read_csv('/workspace/data/git_category_initial.csv')
predata

,Unnamed: 0,category,content,sp3,sp2,sp1
0,0,"[11, 3]",열과 전기가 잘 통해 산업계의 금으로 불리는 희귀 광물인 희토류영구자석 형태로 전기...,NaN,에너지,과학
1,1,"[9, 23, 24]",최근 열린 미국의회 청문회에서 몇몇 앱개발사가 애플과 구글은 플랫폼 영향력 측면에서...,광고,경제,개발
2,2,[29],서효원씨 별세 서동면 서동윤 서동신씨 부친상 김은미씨 시부상 김성덕씨 장인상...,NaN,NaN,etc
3,3,"[23, 13, 5]",비트코인 등 암호화폐 시장이 급락하면서 투자자들이 패닉 상태에 빠졌다 국내 주...,투자,블록체인,경제
4,4,"[24, 17, 23]",웰컴저축은행은 꿈테크 프로젝트 런포드림 영상이 세계 대 국제 광고제인 클리오 어워즈...,경제,미디어,광고
...,...,...,...,...,...,...
1173,1197,[10],퀄컴이 분기 실적 호조를 이어갔다 코로나가 종식되지 않았지만 스마트폰 시장은 예년...,NaN,NaN,통신
1174,1198,[10],아파트 각 가정에 인터넷을 연결해주는 통신 장비입니다 알파벳 U 표시는 이 장비가...,NaN,NaN,통신
1175,1199,[26],이번 주에도 유통식음료 업계는 큰 사건이 없었습니다 지난주 남양유업 사태와 같은 빅...,NaN,NaN,O2O
1176,1201,[26],콘텐츠 커뮤니케이션 기업 유에이와 뉴미디어 컨텐츠 제작기업 비엠미디어가 전통시장의 ...,NaN,NaN,O2O


In [30]:
predata['content'][16] 

'    이광재 더불어민주당 의원이 가상자산 투자자를 보호할 수 없다는 은성수 금융위원장의 발언에 대해 강하게 비판했다 년 당시 박상기 법무부 장관의 거래소 폐쇄 발언까지 언급하며 그때도 틀렸고 지금도 틀렸다고 지적했다    이광재 의원은 일 페이스북 게시글을 통해 그때나 지금이나 시장이 위험하니 막자고 말한다며 저는 이에 대해 생각을 달리한다며 가상자산 시장의 안정성과 투명성을 높여 투자자들을 보호하고 나아가 신산업의 관점에서 접근해야 한다고 강조했다    이 의원은 왜 세대가 가상자산이나 주식에 열광하는지 깊게 고민해야 한다며 그들의 삶이 불안하기 때문에 미래 가능성에 매달리고 있는 것이니 가능성을 현실로 만드는 조건을 마련해 주는 것이 어른들의 역할라고 덧붙였다    그는 가상자산 시장을 산업으로 인정하고 투자할 수 있는 환경을 만들어 달라는 것이라며 객관적 투자정보를 제공해 주고 투명한 시스템을 만들어 건전하게 투자할 수 있게 해달라는 것이라고 분석했다      또 가상자산 시장이 위험하니 막겠다는 접근은 시대착오적이라고 일축하며 테슬라와 골드만삭스는 비트코인에 대규모로 투자했고 페이팔과 마스터카드는 가상자산 유통에 뛰어들었다고 글로벌 동향을 짚었다 이어 이미 세계경제에 깊숙이 파고 들어 우리가 인정하지 않는다고 사라질 것이 아니고 폐쇄한다고 막을 수 있는 것이 아니다고 강조했다    그러면서 이 의원은 가상자산 산업 관련 불법행위 차단 관련제도 정비 미래산업의 측면에서 접근할 것 등 대 정책틀을 제안했다 그는 가상자산 시장을 두고 국무조정실 금융위 기획재정부 한국은행과 과학기술 전문가들이 참여하는 범정부적 테이블을 마련해야 한다고 덧붙였다    한편 앞서 은성수 위원장은 전날 국회 정무위원회 전체회의에서 가상자산 투자자가 늘어난다고 해서 정부가 관심을 갖고 보호해야 한다는 것은 아니다며 가상자산 투자자 보호 방안 마련에 대한 부정적인 입장을 내비친 바 있다   박상기 은성수 이광재 가상자산            저작권자  파이낸셜뉴스 무단 전재재배포 금

In [29]:
predata['category'][16] 

'[13, 23, 5]'

In [ ]:
# 카테고리 별 토크나이징 분류 후 단어 셋
# set 간의 유사도